In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras as keras


MODEL_DIR = f"models/assignment_transformer"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)
plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    load_config["nominal"], max_events=4_000_000
)
X_train, y_train, _, _ = DataProcessor.split_even_odd()
del DataProcessor

2026-01-08 10:45:35.310550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767865535.333189 2795393 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767865535.340634 2795393 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767865535.358900 2795393 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767865535.358920 2795393 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767865535.358922 2795393 computation_placer.cc:177] computation placer alr

In [2]:
TransformerMatcher = Models.SPANetTransformer(data_config, name=r"Transformer")

TransformerMatcher.build_model(
    hidden_dim=32,
    encoder_layers=4,
    decoder_layers=2,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

I0000 00:00:1767865651.678984 2795393 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


I0000 00:00:1767865654.079729 2797550 service.cc:152] XLA service 0x7fd074002a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767865654.079756 2797550 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-01-08 10:47:34.090543: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1767865654.122035 2797550 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1767865654.257096 2797550 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization


2026-01-08 10:47:38.544366: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-08 10:47:39.624624: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


...

Model: "keras_model_wrapper"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ConcatLeptonCharge  │ (None, 2, 4)      │          0 │ lep_input_normal… │
│ (ConcatLeptonCharg… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_mlp   │ (None, 6, 32)     │      1,070 │ jet_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_m… │ (None, 2, 32)     │        975 │ ConcatLeptonChar… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_embedding_mlp   │ (None, 1, 32)     │        740 │ met_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 9, 32)     │          0 │ jet_embedding_ml… │
│ (Concatenate)       │                   │            │ lepton_embedding… │
│                     │                   │            │ met_embedding_ml… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_jet_mask     │ (None, 9)         │          0 │ jet_mask[0][0]    │
│ (ExpandJetMask)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 58,426 (228.24 KB)

 Trainable params: 58,401 (228.13 KB)

 Non-trainable params: 25 (112.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=50,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=8,
            restore_best_weights=True,
            verbose=1,
        ),
    ],
)

Epoch 1/50


2026-01-08 10:48:00.423270: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert


1561/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1642 - loss: 0.1565

2026-01-08 10:48:52.670831: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1642 - loss: 0.1565

2026-01-08 10:49:14.426574: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-08 10:49:17.913872: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 98s 36ms/step - accuracy: 0.1643 - loss: 0.1565 - val_accuracy: 0.4080 - val_loss: 0.1191 - learning_rate: 1.0000e-04
Epoch 2/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.3660 - loss: 0.1258 - val_accuracy: 0.4670 - val_loss: 0.1130 - learning_rate: 1.0000e-04
Epoch 3/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.4125 - loss: 0.1195 - val_accuracy: 0.4926 - val_loss: 0.1111 - learning_rate: 1.0000e-04
Epoch 4/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.4326 - loss: 0.1167 - val_accuracy: 0.4938 - val_loss: 0.1103 - learning_rate: 1.0000e-04
Epoch 5/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.4481 - loss: 0.1149 - val_accuracy: 0.5127 - val_loss: 0.1090 - learning_rate: 1.0000e-04
Epoch 6/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.4617 - loss: 0.1136 - val_accuracy: 0.5429 - val_loss: 0.1076 - learning_rate: 1.0000e-04
Epoch 7/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/

In [ ]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/assignment_transformer/model.keras


: 